# Camera Setup

In [5]:
import cv2
import cv2.aruco as aruco
import os
import numpy as np
import subprocess

# Camera 0 refers to top camera, Camera 1 refers to bottom camera
# Ensure camera port ids are correct
port_ids = [2, 0]
cam0_device = f"/dev/video{port_ids[0]}"
cam1_device = f"/dev/video{port_ids[1]}"

# Ensure proper camera configurations
cam0_focus_value = 35
cam1_focus_value = 75
config_commands = {cam0_device: [
                    f"v4l2-ctl -d {cam0_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam0_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam0_device} -c focus_absolute={cam0_focus_value}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ],
                cam1_device: [
                    f"v4l2-ctl -d {cam1_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam1_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam1_device} -c focus_absolute={cam1_focus_value}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ]
                }

def configure_camera(devices, config_commands):
    for device in devices:

        print(f"Configuring camera on {device}...")

        for command in config_commands[device]:
            subprocess.run(command, shell=True, check=True)

        print("Camera configuration complete!")

# Grab some test images of catheter tip

In [8]:
# Collect images of calibration board in both cameras frames for stereo extrinsic calibration
import cv2
import datetime
import os

output_dir = f"../tip_pose_images"
os.makedirs(output_dir, exist_ok=True)

# Make sure cameras are configures
# configure_camera([cam0_device, cam1_device], config_commands) # Uncomment to use default configs
cap0 = cv2.VideoCapture(port_ids[0], cv2.CAP_V4L2)
cap1 = cv2.VideoCapture(port_ids[1], cv2.CAP_V4L2)
frame_count = 0
while True:
    # Read frames from both cameras
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()

    if not ret0 or not ret1:
        print("Error: One or both frames could not be read.")
        break

    # Display both camera feeds with timestamps
    # timestamp = datetime.now().strftime("%H:%M:%S.%f")
    # cv2.putText(frame1, f"Cam1 - {timestamp}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # cv2.putText(frame2, f"Cam2 - {timestamp}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Combine and display both frames
    combined = cv2.hconcat([frame0, frame1])
    cv2.imshow("Camera 0 (top) + Camera 1 (side)", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord(' '):  # Space key to capture images
        img0_path = f"{output_dir}/cam0_{frame_count}.png"
        img1_path = f"{output_dir}/cam1_{frame_count}.png"
        cv2.imwrite(img0_path, frame0)
        cv2.imwrite(img1_path, frame1)
        print(f"Captured images:\n - {img0_path}\n - {img1_path}")
        frame_count += 1

# Release both cameras and close windows
cap0.release()
cap1.release()
cv2.destroyAllWindows()

Captured images:
 - ../tip_pose_images/cam0_0.png
 - ../tip_pose_images/cam1_0.png
Captured images:
 - ../tip_pose_images/cam0_1.png
 - ../tip_pose_images/cam1_1.png
Captured images:
 - ../tip_pose_images/cam0_2.png
 - ../tip_pose_images/cam1_2.png
Captured images:
 - ../tip_pose_images/cam0_3.png
 - ../tip_pose_images/cam1_3.png


# Segmentation by manual point prompt

In [ ]:
import cv2
import numpy as np
import torch
from segment_anything import sam_model_registry, SamPredictor
from matplotlib import pyplot as plt
import glob

# Mouse callback function
def on_mouse(event, x, y, flags, param):
    global clicked_points, mode
    if event == cv2.EVENT_LBUTTONDOWN:
        label = 1 if mode == 'f' else 0
        clicked_points.append((x, y, label))
        color = (0, 255, 0) if label == 1 else (0, 0, 255)
        cv2.circle(image, (x, y), 5, color, -1)
        cv2.imshow("Select Points (f: foreground, b: background, ESC: done)", image)
        print(f"Clicked point: x={x}, y={y}, label={label}")

# Load SAM model
checkpoint_path = "/home/arclab/repos/segment-anything/checkpoints/sam_vit_b_01ec64.pth"
model_type = "vit_b"
sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
# sam.to("cuda" if torch.cuda.is_available() else "cpu")
sam.to("cpu")
sam_predictor = SamPredictor(sam)

# Loop over images in director
dir = "../tip_pose_images"
cam0_img_path = sorted(glob.glob(f"{dir}/cam0_*.png"))
cam1_img_path = sorted(glob.glob(f"{dir}/cam1_*.png"))
mask_dir = f"{dir}/masks"

for cam_num, (img0_path, img1_path) in enumerate(zip(cam0_img_path, cam1_img_path)):
    # Read images
    img0 = cv2.imread(img0_path)
    img1 = cv2.imread(img1_path)

    # Convert to RGB
    img0_rgb = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

    # Get image dimensions
    h, w, _ = img0.shape

    for image_num, image in enumerate([img0, img1]):

        sam_predictor.set_image(image)

       # Initialize global variables
        clicked_points = []
        mode = 'f'  # Start with foreground mode

        cv2.imshow("Select Points (f: foreground, b: background, ESC: done)", image)
        cv2.setMouseCallback("Select Points (f: foreground, b: background, ESC: done)", on_mouse)

        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 27:  # ESC to exit
                break
            elif key == ord('f'):
                mode = 'f'
                print("Switched to foreground mode.")
            elif key == ord('b'):
                mode = 'b'
                print("Switched to background mode.")

        cv2.destroyAllWindows()

        # Separate points into foreground and background
        fg_coords = np.array([[x, y] for x, y, label in clicked_points if label == 1], dtype=np.float32)
        bg_coords = np.array([[x, y] for x, y, label in clicked_points if label == 0], dtype=np.float32)

        # Predict the mask
        with torch.no_grad():
            masks, scores, logits = sam_predictor.predict(
                point_coords=np.vstack((fg_coords, bg_coords)),
                point_labels=np.array([1] * len(fg_coords) + [0] * len(bg_coords)),
                multimask_output=False
            )

        # Display and save the mask
        mask = masks[0].astype(np.uint8) * 255
        cv2.imshow("Segmented Mask", image)
        cv2.imshow(masks[0], alpha=0.5, cmap='Reds')
        cv2.imwrite(f"{mask_dir}/cam{cam_num}_{image_num}.png", mask)
        cv2.waitKey(0)
        cv2.destroyAllWindows()



/home/arclab/repos/segment-anything/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Switched to foreground mode.
Clicked point: x=313, y=170, label=1
Clicked point: x=311, y=200, label=1
Switched to background mode.
Clicked point: x=314, y=92, label=0
Clicked point: x=436, y=196, label=0
Clicked point: x=289, y=219, label=1
Clicked point: x=288, y=253, label=1
Switched to background mode.
Clicked point: x=288, y=158, label=0
Clicked point: x=345, y=219, label=0
Clicked point: x=323, y=185, label=1
Clicked point: x=354, y=227, label=1
Switched to background mode.
Clicked point: x=312, y=102, label=0
Clicked point: x=367, y=175, label=0
Clicked point: x=286, y=243, label=1
Switched to background mode.
Switched to foreground mode.
Clicked point: x=288, y=264, label=1
Switched to background mode.
Clicked point: x=285, y=175, label=0
Clicked point: x=332, y=254, label=0
Clicked point: x=318, y=167, label=1
Clicked point: x=356, y=227, label=1
Switched to background mode.
Clicked point: x=312, y=96, label=0
Clicked point: x=387, y=154, label=0
Clicked point: x=287, y=236, l